In [14]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, SimpleRNN

# Ch15_RNN과 CNN을 사용해 시퀀스 처리하기

## 순환 신경망(Recurrent Neural Network, RNN) 요약
- `Hands-on Machine Learning`의 내용 중에서 중요하다고 생각되는 내용이나 새롭게 알게 된
내용을 간단하게 요약해서 정리하고자 한다.
- 시간이 충분하지는 않아서 그림과 같은 자세한 설명은 생략하려고 한다.

## 15.1 순환 뉴런과 순환 층
- RNN은 매 타임 스텝 t마다 모든 뉴런은 입력 벡터 $x_{(t)}$와 타임 스텝의 출력 벡터 $y_{(t-1)}$을 받는다.
- 각 순환 뉴런은 입력 벡터 $x_{(t)}$를 위한 가중치 벡터 $w_x$와 출력 벡터 $y_{(t-1)}$를 위한 가중치 벡터 $w_y$를 가진다.
- 이를 순환 층 전체로 생각하면 각각 가중치 행렬 $W_x$, $W_y$가 된다.
- 순환 층 전체의 출력 벡터는 아래와 같은 식으로 표현된다.
- 여기서 $\mathbf{b}$는 편향이고 $\phi$는 활성 함수이다.
> $\mathbf{y}_{(t)}=\phi(\mathbf{W}_{x}^{T}\mathbf{x}_{(t)}+\mathbf{W}_{x}^{T}\mathbf{y}_{(t-1)}+\mathbf{b})$
- 식을 보면, $\mathbf{y}_{(t)}$는 이전 상태의 출력인 $\mathbf{y}_{(t-1)}$도 입력 받기 때문에, 결국 가장 첫 번째 입력인 $\mathbf{x}_{0}$에 대한 값까지도 가지고 있게 된다.
- 첫번째 타임 스텝 t=0에서는 이전 출력이 없으므로 모두 0이라고 가정한다.

### 15.1.1 메모리 셀
- 타입 스텝 t에서 순환 뉴런의 출력은 이전 타임 스텝의 모든 입력에 대한 값이므로 일종의 메모리 형태이다.
- 타임 스텝에 걸쳐서 어떤 상태를 보존하는 신경망의 구성 요소를 메모리 셀(memory cell, 또는 셀)이라고 한다.
- 타임 스텝 t에서의 셀의 상태 $\mathbf{h}_{(t)}$(hidden cell)는 그 타임 스텝의 입력과 이전 타임 스텝의 상태에 대한 함수 $\mathbf{h}_{(t)}=f(\mathbf{h}_{(t-1)}, \mathbf{x}_{(t)})$로 나타낼 수 있다.

### 15.1.2 입력과 출력 시퀀스
- Sequence-to-sequence network: 입력 시퀀스를 받아 출력 시퀀스를 만드는 RNN으로, 시계열 데이터를 예측하는데 유용하다.
- Sequence-to-vector network: 입력 시퀀스를 받아 마지막 출력 벡터만 만드는 RNN이다.
- Vector-to-sequence network: 각 타임 스텝에서 하나의 입력 벡터를 반복해서 입력하고 하나의 시퀀스를 출력하는 RNN으로,
    예를 들어 이미지를 입력하여 이미지에 대한 캡션을 출력하는 RNN이 있다.
- encoder-decoder: 인코더라 부르는 Sequence-to-vector network 뒤에 디코더라 부르는 Vector-to-sequence network를 연결하는 구조로, 문장 번역에 자주 사용된다.
    - 한 언어의 문장을 네트워크에 입력하면, 인코더는 이 문장을 하나의 벡터로 변환하고, 디코더는 이 벡터를 받아 다른 언어의 문장으로 디코딩한다.
    - 문장의 마지막 단어가 번역 시 첫번째 단어에 영향을 주는 경우가 많기 때문에, 인코더-디코더와 같은 이중 RNN은 하나의 Sequence-to-vector network보다 훨씬 더 잘 작동한다고 한다.

## 15.2 RNN 훈련하기
- RNN을 학습시킬 때에는 BPTT(Backpropagation Through Time)을 통해 학습하므로, 그레디언트가 마지막 출력뿐만 아니라 손실 함수를 사용한 모든 출력에 대해 역전파된다.
- 또한 각 타임 스텝마다 같은 매개변수 $\mathbf{W}$와 $\mathbf{b}$가 사용되기 때문에 순전파에서 모두 동일한 가중치가 적용되어 계산이 진행된 후 역전파가 진행되면 모든 타임 스텝에 걸쳐 합산된다.

## 15.3 시계열 예측하기
- 시계열(Time series) 데이터: 타임 스텝마다 하나 이상의 값을 가지는 시퀀스 데이터.
    - 단변량 시계열(Univariate time series): 타임 스텝마다 하나의 값을 가지는 데이터.
    - 다변량 시계열(Multivariate time series): 타임 스텝마다 여러 값을 가지는 데이터.
- 시계열을 다룰 때 입력 특성은 일반적으로 [배치 크기, 타임 스텝 수, 차원 수] 크기의 3D 배열로 나타낸다고 한다.
- 따라서 단변량 시계열의 경우 차원 수는 1이 되고, 다변량 시계열의 경우 차원 수가 1 이상이 된다.

In [3]:
# 시계열 생성해주는 함수이다.
# batch_size만큼 n_steps 길이의 여러 시계열을 만든다.
# [배치 크기, 타임 스텝 수, 1] 크기의 넘파이 배열을 리턴하므로 이 시계열 데이터는 단변량 데이터이다.
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # 사인 곡선 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + 사인 곡선 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + 잡음
    return series[..., np.newaxis].astype(np.float32)

In [4]:
# 위의 함수를 이용하여 훈련셋, 검증셋, 테스트셋을 생성한다.
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

### 15.3.1 기준 성능
- RNN과 비교할 비교군으로 사용할 모델들을 생성한다.
1. 순진한 예측(Naive forecasting): 각 시계열의 마지막 값을 그대로 예측
    - 이렇게 하는 것도 높은 확률이 될 수 있다.

In [23]:
y_pred = X_valid[:, -1]
mse = np.mean(tf.keras.losses.mean_squared_error(y_valid, y_pred))
print(f"mse: {mse}")

mse: 0.02011820673942566


2. MLP(완전 연결층) 사용
    - 검증셋에 대한 mse를 확인해보면, 순진한 예측보다는 좋은 결과를 출력하는 것을 알 수 있다.

In [24]:
model = Sequential([
    Flatten(input_shape=[50, 1]),
    Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.MeanSquaredError())

history = model.fit(X_train, y_train, epochs=20, verbose=False)
mse = model.evaluate(X_valid, y_valid, verbose=2)
print(f"mse: {mse}")

2000/1 - 0s - loss: 0.0030
mse: 0.0037209187019616365
